In [1]:
import os

In [2]:
os.getcwd()

'd:\\Learnings\\GenAI\\KNAC\\myfiles\\projects\\Agentic_Trading_Chatbot\\notebook'

In [3]:
os.chdir("..")

In [4]:
os.getcwd()

'd:\\Learnings\\GenAI\\KNAC\\myfiles\\projects\\Agentic_Trading_Chatbot'

# contants

In [ ]:
from dataclasses import dataclass 
from trading_bot.utils import load_yaml 


CONFIG=load_yaml("config/config.yaml")

@dataclass 
class ToolsConstants:
    RETRIEVER_TOP_K=CONFIG.TOOLS.RETRIEVER.TOP_K
    RETRIEVER_SCORE_THRESHOLD=CONFIG.TOOLS.RETRIEVER.SCORE_THRESHOLD
    TAVILY_MAX_RESULTS=CONFIG.TOOLS.TAVILY.MAX_RESULTS

In [10]:
print("RETRIEVER_TOP_K:", ToolsConstants.RETRIEVER_TOP_K)
print("RETRIEVER_SCORE_THRESHOLD:", ToolsConstants.RETRIEVER_SCORE_THRESHOLD)
print("TAVILY_MAX_RESULTS:", ToolsConstants.TAVILY_MAX_RESULTS)

RETRIEVER_TOP_K: 3
RETRIEVER_SCORE_THRESHOLD: 0.5
TAVILY_MAX_RESULTS: 5



# entity

In [13]:
from dataclasses import dataclass 

@dataclass 
class Tools:
    RETRIEVER_TOP_K:int
    RETRIEVER_SCORE_THRESHOLD:float
    TAVILY_MAX_RESULTS:int

# configuration

In [14]:
from dataclasses import dataclass 

@dataclass 
class ToolsConfig:
    RETRIEVER_TOP_K=int(ToolsConstants.RETRIEVER_TOP_K)
    RETRIEVER_SCORE_THRESHOLD=float(ToolsConstants.RETRIEVER_SCORE_THRESHOLD)
    TAVILY_MAX_RESULTS=int(ToolsConstants.TAVILY_MAX_RESULTS)

In [16]:
print("RETRIEVER_TOP_K:", ToolsConfig.RETRIEVER_TOP_K)
print(type(ToolsConfig.RETRIEVER_TOP_K))
print("RETRIEVER_SCORE_THRESHOLD:", ToolsConfig.RETRIEVER_SCORE_THRESHOLD)
print(type(ToolsConfig.RETRIEVER_SCORE_THRESHOLD))
print("TAVILY_MAX_RESULTS:", ToolsConfig.TAVILY_MAX_RESULTS)
print(type(ToolsConfig.TAVILY_MAX_RESULTS))

RETRIEVER_TOP_K: 3
<class 'int'>
RETRIEVER_SCORE_THRESHOLD: 0.5
<class 'float'>
TAVILY_MAX_RESULTS: 5
<class 'int'>


# schema

In [17]:
from pydantic import BaseModel


class RagToolSchema(BaseModel):
    question:str 
class QuestionRequest(BaseModel):
    question: str

# components

In [ ]:
from langchain_community.tools.polygon.financials import PolygonFinancials
from langchain_community.utilities.polygon import PolygonAPIWrapper
from trading_bot.utils import get_vector_store, load_embeddings
from langchain_community.tools import TavilySearchResults
from trading_bot.exception import CustomException
from trading_bot.entity import DataIngestion
from trading_bot.logger import logging
from dataclasses import dataclass
from langchain.tools import tool
from dotenv import load_dotenv
import sys


@dataclass
class ToolsComponents:
    __data_ingestion_config:DataIngestion
    __tools_config:Tools

    def __call__(self, get_tools:bool=None) -> dict[tool]:
        """post init to initalize tools

        Args:
            get_tools (bool, optional): if not provided error will be raised. Defaults to None.        

        Returns:
            dict[tool]: {
            "retriever_tool": retriever_tool,
            "tavily_tool": tavily_tool,
            "financials_tool": financials_tool
            }
        """
        try:
            if get_tools==None:
                raise ValueError("Value should only be boolean")
            
            if get_tools:
                # retriever
                @tool(args_schema=RagToolSchema)
                def retriever_tool(question):
                    """Retrieves n numbers of data from database on the basis of similarity search
                    """
                    # load api keys
                    load_status=load_dotenv()
                    logging.info(f"load_dotenv: {{{load_status}}}")

                    # load embeddig model
                    embeddings=load_embeddings(self.__data_ingestion_config.GOOGLE_EMBEDDING_MODEL_NAME, 
                                                self.__data_ingestion_config.HUGGINGFACE_EMBEDDING_MODEL_NAME)
                    try:
                        embeddings_model_name=embeddings.model
                    except:
                        embeddings_model_name=embeddings.model_name
                    logging.info(f"loaded embeddings model {{{embeddings_model_name}}}")
                    
                    # connect with vector store
                    vector_store = get_vector_store(pinecone_api_key=os.getenv("PINECONE_API_KEY"), 
                                                    index_name= self.__data_ingestion_config.PINECONE_INDEX_NAME, 
                                                    embeddings=embeddings)
                    logging.info("loaded vector store")

                    retriever = vector_store.as_retriever(
                        search_type="similarity_score_threshold",
                        search_kwargs={
                            "k": self.__tools_config.RETRIEVER_TOP_K , 
                            "score_threshold": self.__tools_config.RETRIEVER_SCORE_THRESHOLD
                        }
                    )
                    return retriever.invoke(question)
                
                # tavily
                tavily_tool = TavilySearchResults(
                max_results=self.__tools_config.TAVILY_MAX_RESULTS,
                search_depth="advanced",
                include_answer=True,
                include_raw_content=True,
                )

                # polygon
                api_wrapper = PolygonAPIWrapper()
                financials_tool = PolygonFinancials(api_wrapper=api_wrapper)
                logging.info("all tools collected")
                
                return {
                    "retriever_tool": retriever_tool,
                    "tavily_tool": tavily_tool,
                    "financials_tool": financials_tool
                    }

        except Exception as e:
            logging.info(e)
            raise CustomException(e, sys)

# pipeline

In [ ]:
from trading_bot.configuration import DataIngestionConfig


class QueryPipeline:

    def run(self):
        # tools
        tools_obj=ToolsComponents(DataIngestionConfig, ToolsConfig)
        tools_dict=tools_obj(get_tools=True)
        tools=[
            tools_dict["retriever_tool"], 
            tools_dict["tavily_tool"], 
            tools_dict["financials_tool"]
        ]

In [27]:
tools_obj=ToolsComponents(DataIngestionConfig, ToolsConfig)
tools_obj

ToolsComponents(_ToolsComponents__data_ingestion_config=<class 'trading_bot.configuration.DataIngestionConfig'>, _ToolsComponents__tools_config=<class '__main__.ToolsConfig'>)

In [28]:
tools=tools_obj(get_tools=True)
tools

{'retriever_tool': StructuredTool(name='retriever_tool', description='Retrieves n numbers of data from database on the basis of similarity search', args_schema=<class '__main__.RagToolSchema'>, func=<function ToolsComponents.__call__.<locals>.retriever_tool at 0x000001C74D562680>),
 'tavily_tool': TavilySearchResults(include_answer=True, include_raw_content=True, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))),
 'financials_tool': PolygonFinancials(api_wrapper=PolygonAPIWrapper(polygon_api_key='kgX5WOExrCZ52VFxMuldtUHL1pnTnLRt'))}

In [31]:
tools["retriever_tool"]

StructuredTool(name='retriever_tool', description='Retrieves n numbers of data from database on the basis of similarity search', args_schema=<class '__main__.RagToolSchema'>, func=<function ToolsComponents.__call__.<locals>.retriever_tool at 0x000001C74D562680>)

In [32]:
tools["tavily_tool"]

TavilySearchResults(include_answer=True, include_raw_content=True, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))

In [33]:
tools["financials_tool"]

PolygonFinancials(api_wrapper=PolygonAPIWrapper(polygon_api_key='kgX5WOExrCZ52VFxMuldtUHL1pnTnLRt'))